In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoremiscv2
from nba_api.stats.endpoints import boxscorefourfactorsv2
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoreusagev2
import matplotlib.pyplot as plt
from keras import backend as K
from sklearn import preprocessing
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor,BaggingClassifier, RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc
from sklearn.preprocessing import MinMaxScaler

In [ ]:
reg15 = pd.read_csv('../../reg15.csv')
misc15 = pd.read_csv('../../misc15.csv')
adv15 = pd.read_csv('../../adv15.csv')
four15 = pd.read_csv('../../four15.csv')
reg16 = pd.read_csv('../../reg16.csv')
misc16 = pd.read_csv('../../misc16.csv')
adv16 = pd.read_csv('../../adv16.csv')
four16 = pd.read_csv('../../four16.csv')
reg17 = pd.read_csv('../../regular_1718.csv')
misc17 = pd.read_csv('../../misc17.csv')
adv17 = pd.read_csv('../../advanced17.csv')
four17 = pd.read_csv('../../four17.csv')
reg18 = pd.read_csv('../../reg18.csv')
misc18 = pd.read_csv('../../misc18.csv')
adv18 = pd.read_csv('../../advanced18.csv')
four18 = pd.read_csv('../../four18.csv')
reg19 = pd.read_csv('../../reg19.csv')
misc19 = pd.read_csv('../../misc19.csv')
adv19 = pd.read_csv('../../adv19.csv')
four19 = pd.read_csv('../../four19.csv')

team_ids = reg16.TEAM_ID.value_counts().index

total15 = get_features(reg15,misc15,adv15,four15)
total16 = get_features(reg16,misc16,adv16,four16)
total17 = get_features(reg17,misc17,adv17,four17)
total18 = get_features(reg18,misc18,adv18,four18)
total19 = get_features(reg19,misc19,adv19,four19)

features = [total16,total17,total18]
test_features = [total15]

vegas15 = pd.read_csv('archive/2015-16/vegas.txt',sep = ',')
vegas16 = pd.read_csv('archive/2016-17/vegas.txt',sep = ',')
vegas17 = pd.read_csv('archive/2017-18/vegas.txt',sep = ',')
vegas18 = pd.read_csv('archive/2018-19/vegas.txt',sep = ',')

targets = [vegas16,vegas17,vegas18]
test_targ = [vegas15]

sixteen_eighteen = get_targets_ML(features,targets)
test = get_targets_ML(test_features,test_targ)

X_train = sixteen_eighteen[0].iloc[0:4500,:]
X_test = sixteen_eighteen[0].iloc[4500:,:]

sixteen_eighteen[1]['prob_a'] = 1/(sixteen_eighteen[1]['odds_a'])
y = sixteen_eighteen[1][['prob_a','a_win']]
y_train = y.iloc[0:4500,:]
y_test = y.iloc[4500:,:]

nba_teams = ['Cleveland Cavaliers','Boston Celtics','Golden State Warriors','Houston Rockets','Utah Jazz'
             ,'Philadelphia 76ers','Toronto Raptors','New Orleans Pelicans','Washington Wizards','Miami Heat'
            ,'Milwaukee Bucks','Indiana Pacers','Oklahoma City Thunder','San Antonio Spurs','Portland Trail Blazers',
             'Minnesota Timberwolves','Chicago Bulls','Dallas Mavericks','Sacramento Kings','Los Angeles Lakers'
            ,'Orlando Magic','Denver Nuggets','LA Clippers','New York Knicks','Memphis Grizzlies','Detroit Pistons'
            ,'Charlotte Hornets','Atlanta Hawks','Phoenix Suns','Brooklyn Nets']



In [2]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['GAME_ID'])
    # Filter out any row that is joined to itself.
    
    for columnName in joined:
        if(columnName == 'TEAM_ID_A_x'):
            joined = joined.rename(columns = {'TEAM_ID_A_x':'TEAM_ID_A'})
        
    for columnName in joined:
        if(columnName == 'TEAM_ID_B_x'):
            joined = joined.rename(columns = {'TEAM_ID_B_x':'TEAM_ID_B'})  
        
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [3]:
def combine_vegas_games(df):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['GameId'])
    # Filter out any row that is joined to itself.
    
    result = joined[joined.TeamId_A != joined.TeamId_B]
    # Take action based on the keep_method flag.
    return result

In [5]:
def remove_unnamed(frame):
    for columnName in frame:
        if(columnName[0:7] == 'Unnamed'):
            frame = frame.drop(columns = [columnName])
    return frame

In [6]:
def pull_reg_season(start_year, start_date, end_date):
    reg = pd.DataFrame()
    for i in team_ids:
        games_base = leaguegamefinder.LeagueGameFinder(team_id_nullable = i)
        games_xxxx = games_base.get_data_frames()[0]
        games_xxxx = games_xxxx[games_xxxx.TEAM_NAME.isin(nba_teams)]
        regular_xxxx = games_xxxx[(games_xxxx.GAME_DATE > start_date) & (games_xxxx.GAME_DATE < end_date)]
        reg = reg.append(regular_xxxx)
        time.sleep(30)
    reg = reg.sort_values('GAME_ID')
    return reg

In [34]:
def pull_stats(api, index, ids):
    box = pd.DataFrame()
    subset1 = ids[0:308]
    subset2 = ids[308:616]
    subset3 = ids[616:924]
    subset4 = ids[924:1231]
    subset_list = [subset1, subset2, subset3, subset4]
    for i in subset_list:
        for j in i:
            print(j)
            row = api(game_id = j)
            row = row.get_data_frames()[index]
            row['GAME_ID'] = j
            row = combine_team_games(row, keep_method = None)
            box = box.append(row)
        time.sleep(60)
    return box
        

            
                
            
    

In [35]:
def pull_stats_slow(api, index, ids):
    box = pd.DataFrame()
    subset1 = ids[0:77]
    subset2 = ids[77:154]
    subset3 = ids[154:231]
    subset4 = ids[231:308]
    subset5 = ids[308:385]
    subset6 = ids[385:462]
    subset7 = ids[462:539]
    subset8 = ids[539:616]
    subset9 = ids[616:693]
    subset10 = ids[693:770]
    subset11 = ids[770:847]
    subset12 = ids[847:924]
    subset13= ids[924:1001]
    subset14 = ids[1001:1078]
    subset15 = ids[1078:1155]
    subset16 = ids[1155:1231]
    subset_list = [subset1, subset2, subset3, subset4,subset5, subset6, subset7, subset8,
                  subset9, subset10, subset11, subset12,subset13, subset14, subset15, subset16]
    counter = 0
    for i in subset_list:
        for j in i:
            row = api(game_id = j)
            row = row.get_data_frames()[index]
            row['GAME_ID'] = j
            row = combine_team_games(row, keep_method = None)
            box = box.append(row)
            print(counter)
            counter = counter + 1
            time.sleep(1)
        time.sleep(150)
    return box

In [36]:
def pull_stats_super_slow(api, index, reg_season):
    box = pd.DataFrame()
    reg_season = reg_season.drop_duplicates(subset = ['GAME_ID'])
    ids = reg_season['GAME_ID']
    subset1 = ids[0:308]
    subset2 = ids[308:616]
    subset3 = ids[616:924]
    subset4 = ids[924:1231]
    subset_list = [subset1, subset2, subset3, subset4]
    for i in subset_list:
        for j in i:
            row = api(game_id = j)
            row = row.get_data_frames()[index]
            row['GAME_ID'] = j
            row = combine_team_games(row, keep_method = None)
            box = box.append(row)
            time.sleep(10)
    return box

In [37]:
def reg_clean(reg):
    if(('GAME_ID')[0] == '0'):
        reg['GAME_ID'] = reg['GAME_ID'].str[2:].astype(int)
    reg = reg.sort_values('GAME_ID')
    reg = combine_team_games(reg,keep_method=None)
    return reg
    
    

In [38]:
def home_win(WL_A):
    if(WL_A == 'W'):
        return([1,0,0])
    else:
        return([0,1,0])

In [39]:
def adv_clean(advanced):
    advanced = advanced.drop(columns = ['TEAM_ID_B','TEAM_ABBREVIATION_A','TEAM_ABBREVIATION_B'
                                   ,'TEAM_CITY_A','TEAM_CITY_B','TEAM_NAME_A','TEAM_NAME_B','MIN_A'])
    advanced = advanced.sort_values('GAME_ID')
    return advanced
    


In [40]:
def traditional_clean(traditional):
    bench = traditional.drop_duplicates(subset = ['GAME_ID','TEAM_NAME_A'], keep = 'last')
    starters = traditional.drop_duplicates(subset = ['GAME_ID','TEAM_NAME_A'], keep = 'first')
    bench = bench.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'STARTERS_BENCH_A', 'MIN_A','TEAM_ID_B', 'TEAM_NAME_B',
       'TEAM_ABBREVIATION_B', 'TEAM_CITY_B', 'STARTERS_BENCH_B', 'MIN_B'])
    starters = starters.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'STARTERS_BENCH_A', 'MIN_A','TEAM_ID_B', 'TEAM_NAME_B',
       'TEAM_ABBREVIATION_B', 'TEAM_CITY_B', 'STARTERS_BENCH_B', 'MIN_B'])
    for columnName in bench:
        if((columnName == 'GAME_ID') | (columnName == 'TEAM_ID_A')):
            continue
        new_col_name = columnName + '_bench'
        bench[new_col_name] = bench[columnName]
        bench = bench.drop(columns = [columnName])
    for columnName in starters:
        if((columnName == 'GAME_ID') | (columnName == 'TEAM_ID_A')):
            continue
        new_col_name = columnName + '_starters'
        starters[new_col_name] = starters[columnName]
        starters = starters.drop(columns = [columnName])
        
    merged = pd.merge(starters,bench,left_on = 'GAME_ID',right_on = 'GAME_ID')
    merged = merged.drop_duplicates(subset = ['GAME_ID','TEAM_ID_A_x'])
    merged = merged.drop(columns = ['TEAM_ID_A_y'])
    merged = merged.rename(columns={"TEAM_ID_A_x": "TEAM_ID_A"})
    return(merged)
    

In [41]:
def four_clean(four):
    four = four.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'MIN_A','TEAM_ABBREVIATION_B',
       'TEAM_CITY_B', 'MIN_B','OREB_PCT_A','TM_TOV_PCT_A','EFG_PCT_A'])
    four = four.sort_values('GAME_ID')
    return four

In [42]:
def usage_clean(usage):
    return usage
    

In [43]:
def misc_clean(misc):
    misc = misc.drop(columns = ['TEAM_NAME_A', 'TEAM_ABBREVIATION_A',
       'TEAM_CITY_A', 'MIN_A'])
    return misc

In [44]:
def get_minutes(matchupmisc):
    matchupmisc = matchupmisc.reset_index(drop=True)
    matchupmisc = matchupmisc.sort_values('GAME_ID')
    matchupmisc['minutes'] = (matchupmisc['MIN_A'].str[0:3].astype(int))/5
    minutes =  matchupmisc['minutes']
    return minutes
    
    

In [45]:
def per_minute_stats(frame, minutes):
    frame = frame.reset_index(drop = True)
    features = frame.drop(columns = ['GAME_ID'])
    game_id = frame['GAME_ID'].reset_index(drop = True)
    team_id = frame['TEAM_ID_A'].reset_index(drop=True)
    for columnName in features:
        if(columnName[-1] == 'B'):
            features = features.drop(columns = [columnName])
    for columnName in features:
        if(columnName[-1] == 'A'):
            features = features.rename(columns={columnName: columnName[0:-2]})
    features['minutes'] = minutes
    cumulative = features.groupby('TEAM_ID').cumsum()
    for columnName in cumulative:
        if((columnName == 'minutes') | (columnName == 'TEAM_ID')):
            continue
        new_col_name = columnName + '_per_minute'
        features[new_col_name] = ((cumulative[columnName] - features[columnName]) / (cumulative['minutes'] - features['minutes']))
        features = features.drop(columns = [columnName])
    features = features.reset_index(drop= True)
    features= features.drop(columns = ['minutes'])
    features['TEAM_ID'] = team_id
    features['GAME_ID'] = game_id
    return combine_team_games(features, keep_method = None)
    
        
    

In [46]:
def cum_stats(frame):
    frame = frame.reset_index(drop = True)
    features = frame.drop(columns = ['GAME_ID'])
    game_id = frame['GAME_ID'].reset_index(drop = True)
    team_id = frame['TEAM_ID_A'].reset_index(drop=True)
    for columnName in features:
        if(columnName[-1] == 'B'):
            features = features.drop(columns = [columnName])
    for columnName in features:
        if(columnName[-1] == 'A'):
            features = features.rename(columns={columnName: columnName[0:-2]})
    features = features.reset_index(drop = True)
    for columnName in features:
        if(columnName == 'TEAM_ID'):
            continue
        new_col_name = columnName + '_median'
        features[new_col_name] = features.groupby('TEAM_ID')[columnName].apply(lambda x: x.shift().expanding().median())
        features = features.drop(columns = [columnName])
    features = features.reset_index(drop= True)
    features['TEAM_ID'] = team_id
    features['GAME_ID'] = game_id
    features = features.reset_index(drop = True)
    return combine_team_games(features, keep_method = None)

In [47]:
def reg_impute(reg_cleaned, minutes):
    per_minute = reg_cleaned[['TEAM_ID_A', 'GAME_ID', 'PTS_A', 'FGM_A', 'FGA_A',
                      'FG3M_A','FG3A_A','FTM_A', 'FTA_A','OREB_A', 'DREB_A',
       'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PLUS_MINUS_A']]
    
    cum = reg_cleaned[['TEAM_ID_A', 'GAME_ID','FG_PCT_A','FG3_PCT_A','FT_PCT_A']]
    
    per_minute = per_minute_stats(per_minute, minutes)
    cum = cum_stats(cum)
    merged = pd.merge(cum,per_minute,left_on = ['GAME_ID','TEAM_ID_A'],right_on=['GAME_ID','TEAM_ID_A'])
    #merged = merged.drop_duplicates(subset = ['TEAM_ID_A_x','GAME_ID','TEAM_ID_B_x'])
    merged = merged.rename(columns = {'TEAM_ID_A_x':'TEAM_ID'})
    for columnName in merged:
        if(columnName[-1] == 'B'):
            merged = merged.drop(columns = [columnName])
    for columnName in merged:
        if(columnName[-1] == 'A'):
            merged = merged.rename(columns={columnName: columnName[0:-2]})
    merged = merged.drop(columns = ['TEAM_ID_B_x','TEAM_ID_B_y'])
    merged = combine_team_games(merged,keep_method = None)
    #return merged
    #merged['GAME_ID'] = merged['GAME_ID'].str[2:].astype(int)
    return merged


In [48]:
def get_62(frame):
    return frame.iloc[600:,:]

In [49]:
def four_impute(four_cleaned):
    cum = cum_stats(four_cleaned)
    #cum['GAME_ID'] = cum['GAME_ID'].str[2:].astype(int)
    return cum

In [50]:
def adv_impute(advanced_cleaned):
    cum = cum_stats(advanced_cleaned)
    return cum

In [51]:
def merger(reg_imputed, advanced_imputed, four_imputed):
    merged = pd.merge(reg_imputed,advanced_imputed, left_on = ['GAME_ID','TEAM_ID_A'],right_on = ['GAME_ID','TEAM_ID_A'])
    merged = pd.merge(merged,four_imputed, left_on = ['GAME_ID','TEAM_ID_A'],right_on = ['GAME_ID','TEAM_ID_A'])
    return get_62(merged)

In [52]:
def dataload_clean(year,start_date,end_date):
    reg = pull_reg_season(year,start_date,end_date)
    misc = pull_stats_slow(boxscoremiscv2.BoxScoreMiscV2,1,reg)
    minutes = get_minutes(misc)
    advanced = pull_stats_slow(boxscoreadvancedv2.BoxScoreAdvancedV2,1,reg)
    four = pull_stats_slow(boxscorefourfactorsv2.BoxScoreFourFactorsV2,1,reg)
    merged = merger(reg_impute(reg_clean(reg), minutes),advanced_impute(advancedboxscore_clean(advanced)),
                    four_impute(four_factors_clean(four)))
    merged.to_csv(year + '.csv')
    return merged

In [53]:
def dataloader(year,start_date,end_date):
    reg = pull_reg_season(year,start_date,end_date)
    reg.to_csv('reg' + year + '.csv')
    misc = pull_stats(boxscoremiscv2.BoxScoreMiscV2,1,get_ids(reg))
    misc.to_csv('misc' + year + '.csv')
    adv = pull_stats(boxscoreadvancedv2.BoxScoreAdvancedV2,1,get_ids(reg))
    adv.to_csv('adv' + year + '.csv')
    four = pull_stats(boxscorefourfactorsv2.BoxScoreFourFactorsV2,1,get_ids(reg))
    four.to_csv('four' + year + '.csv')
    return[reg,misc,adv,four]
    

In [54]:
def get_ids(reg):
    unique = reg.drop_duplicates('GAME_ID')
    return unique['GAME_ID']

In [55]:
def add_zeros(game_id):
    new_id = '00' + (str(game_id))
    return new_id

In [10]:
def get_features(reg,misc,adv,four):
    reg2 = reg_impute(reg_clean(reg),get_minutes(misc))
    adv2 = adv_impute(adv_clean(adv))
    four2 = four_impute(four_clean(four))
    merged = merger(reg2,adv2,four2)
    #merged = feature_clean(merged)
    merged = remove_unnamed(merged)
    return merged

In [11]:
def odds_convert(american_lines):
    odds = []
    for i in american_lines:
        if(i < 0):
            odds.append((i -100)/i)
        else:
            odds.append((i+100)/100)
    return odds

In [12]:
def feature_clean(features):
    for columnName in features:
        if ((columnName[0:4] == 'TEAM') | (columnName[0:4] == 'GAME')):
            features = features.drop(columns = [columnName])
    return features

In [13]:
def get_targets_OU(features, targets):
    df = pd.DataFrame()
    for i in range(len(features)):
        year = features[i]
        vegas = targets[i]
        vegas = vegas[['Pinnacle_Line_OU','Pinnacle_Odds_OU','Total','GameId']]
        merged = pd.merge(year,vegas, left_on = 'GAME_ID',right_on = 'GameId')
        merged = merged.drop_duplicates(subset = ['GAME_ID','TEAM_ID_A'])
        merged = merged.dropna()
        df = df.append(merged)
    y = df[['Pinnacle_Line_OU','Total']]
    #odds = y['Pinnacle_Odds_OU']
    #new = odds_convert(odds)
    #y['Pinnacle_Odds_OU'] = new
    x = df.drop(columns = ['Pinnacle_Line_OU','Pinnacle_Odds_OU','Total','GameId'])
    x = feature_clean(x)
    return[x,y]
        

In [14]:
def get_targets_ML(features, targets):
    df = pd.DataFrame()
    for i in range(len(features)):
        year = features[i]
        vegas = targets[i]
        vegas = combine_vegas_games(vegas)
        vegas = vegas[['Pinnacle_ML_A','Pinnacle_ML_B','Result_A','GameId','TeamId_A']]
        merged = pd.merge(year,vegas, left_on = ['GAME_ID','TEAM_ID_A'],right_on = ['GameId','TeamId_A'])
        merged = merged.drop_duplicates(subset = ['GAME_ID','TEAM_ID_A'])
        merged = merged.dropna()
        df = df.append(merged)
    y = df[['Result_A','Pinnacle_ML_A','Pinnacle_ML_B','GameId']]
    y['Odds_A'] = odds_convert(y['Pinnacle_ML_A'])
    y['Odds_B'] = odds_convert(y['Pinnacle_ML_B'])
    y['y_true'] = y.apply(lambda row : get_y_true(row['Result_A'],
                     row['Odds_A'], row['Odds_B']), axis = 1)
    x = df.drop(columns = ['Pinnacle_ML_A','Pinnacle_ML_B','Result_A','GameId','TeamId_A'])
    x = feature_clean(x)
    y = y[['y_true']]
    y2 = pd.DataFrame(y['y_true'].to_list(), columns = ['a_win', 'b_win', 'no_bet','odds_a','odds_b'])
    return[x,y2]
    

In [15]:
def get_y_true(result_a, ml_a,ml_b):
    if(result_a == 'W'):
        return [1,0,0,ml_a,ml_b]
    else:
        return [0,1,0,ml_a,ml_b]
    

In [16]:
def OU_accuracy(X_test, model, y_test,thresh):
    y_test['predictions'] = model.predict(X_test)
    y_test['diff'] = abs(y_test['Pinnacle_Line_OU'] - y_test['predictions'])
    y_test = y_test[y_test['diff'] > thresh]
    counter = 0
    for i in range(y_test.shape[0]):
        ou = y_test.iloc[i,0]
        total = y_test.iloc[i,1]
        predictions = y_test.iloc[i,2]
        if(ou > total):
            if(predictions < ou):
                counter = counter + 1
        if(ou < total):
            if(predictions > ou):
                counter = counter + 1
    return(counter/y_test.shape[0])
            
            
            
    
    
    

In [293]:
def decorrelation_loss(y_true, y_pred):
    p = y_true[:,0:1]
    result = y_true[:,1:2]
    return K.sqrt(K.mean(K.square(result - y_pred) - (0.6 * K.square(p - y_pred))))

In [18]:
def profit_loss(y_true,y_pred):
    home_win = y_true[:,0:1]
    away_win = y_true[:,1:2]
    no_bet = y_true[:,2:3]
    odds_a = y_true[:,3:4]
    odds_b = y_true[:,4:5]
    gain_loss_vector = K.concatenate([(home_win * (odds_a - 1)) + ((1-home_win)* -1),
                                     (away_win * (odds_b - 1)) + ((1-away_win) * -1),
                                     K.zeros_like(odds_a)],axis=1)
    return( -1 * (K.mean(K.sum(gain_loss_vector * y_pred, axis=1))))
    

In [19]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
    

In [20]:
def OU_features(features,corr,thresh):
    size = corr.shape[0]-1
    features2 = features.copy(deep=False)
    for columnName in corr:
        if (corr[columnName][size] < thresh):
            features2 = features2.drop(columns = [columnName])
    return features2
            

In [133]:
def get_bets(pred_odds):
    pred_odds['prob_a'] = 1 / (pred_odds['odds_a'])
    pred_odds['bets'] = pred_odds.apply(lambda row : get_bet(row['predictions'],row['prob_a']), axis = 1)
    return pred_odds
    

In [134]:
def get_bet(pred,odds):
    if((pred-odds) > 0):
        return(pred-odds)
    else:
        return 0

In [23]:
def get_profits(preds):
    preds['profit'] = preds.apply(lambda row: get_profit(row['odds_a'],row['bets'],row['a_win']),axis=1)
    return(preds['profit'].sum())

In [89]:
def filter_features(features, features_list):
    features2 = features.copy(deep=False)
    for columnName in features2:
        if(columnName in features_list):
            continue
        else:
            features2 = features2.drop(columns = [columnName])
    return features2
    

In [24]:
def get_profit(odds,bet,result):
    if(result == 1):
        return((bet*odds) - bet)
    if(result == 0):
        return(-1 * bet)

In [452]:
def profit(model, X_test, y_test):
    pred = model.predict(X_test)
    odds = y_test['odds_a']
    result =  y_test['a_win']
    test = pd.DataFrame()
    test['odds_a'] = odds
    test['a_win'] = result
    test['predictions'] = pred
    get_bets(test)
    profit = get_profits(test)
    print('total profit:  ' + str(profit))
    print('total bet:  ' + str(test['bets'].sum()))
    print('percent_return:  ' + str((100 * (profit/test['bets'].sum()))))

In [25]:
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

In [372]:
def get_model(input_dim, output_dim, base=1000, multiplier=0.25, p=0.2,lr = 0.0001):
    inputs = Input(shape=(input_dim,))
    l = BatchNormalization()(inputs)
    l = Dropout(p)(l)
    n = base
    l = Dense(n, activation='relu')(l)
    l = BatchNormalization()(l)
    l = Dropout(p)(l)
    n = int(n * multiplier)
    l = Dense(n, activation='relu')(l)
    l = BatchNormalization()(l)
    l = Dropout(p)(l)
    n = int(n * multiplier)
    l = Dense(n, activation='relu')(l)
    outputs = Dense(output_dim, activation='sigmoid')(l)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr), loss=decorrelation_loss)
    return model

In [440]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import kerastuner as kt
def train(model,X_train,y_train,val_split,ep,bs,nodes):
    history = model.fit(X_train,y_train,validation_split = val_split,epochs = ep, batch_size = bs)

In [450]:

train(get_model(94,1,10000,0.5,0.4,0.00001),X_train,y_train,0.2,2,50)

    

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
72/72 [==============================] - ETA: 0s - loss: 0.4863WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f827067fc20> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full outp

In [453]:
profit(model,X_test,y_test)

32/32 [==============================] - 1s 25ms/step
total profit:  0.6439284256201341
total bet:  21.50081939904875
percent_return:  2.9949017926666697


/opt/anaconda3/envs/Python_3713/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/anaconda3/envs/Python_3713/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/Python_3713/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste